In [79]:
import spotipy
import spotipy.util as util
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup as bs
import datetime
from datetime import timedelta, date
import glob
from os import listdir
import numpy as np
import pymongo
import csv
import json
import sys, getopt, pprint
from pymongo import MongoClient
import types
import itertools

In [15]:
def current_user_recently_played(self, limit=50):
    return self._get('me/player/recently-played', limit=limit)

In [16]:
scope="user-read-recently-played user-read-private user-top-read user-read-currently-playing"

In [17]:
if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print ("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [18]:
token = util.prompt_for_user_token(username,scope,
                           client_id='e67d7b1bc1ee460bbef7484ee3f601ef',
                           client_secret='ba0087b5f4ab4fb8ab88be1850854538',
                           redirect_uri='https://google.com')



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=e67d7b1bc1ee460bbef7484ee3f601ef&response_type=code&redirect_uri=https%3A%2F%2Fgoogle.com&scope=user-read-currently-playing+user-read-private+user-read-recently-played+user-top-read in your browser


Enter the URL you were redirected to: https://www.google.com/?code=AQB8l6gx2EmqVa0ubOmvwX45v9qD9__Voym4vy0NcFEv6eal0q2lVo2a-NCtq4OieOZ68ETcdX_8ZYXD-5Kj6oHkObFGEXSeIbYZcczkMf1Z6MUqMYmKjGfJbWF-DSF5qgHEWx13UpuaAwCXbohxejGv2PnBGW5Wfn7lYeweOymVOz-jrLhZVs0cp9xuj-lEMRNa8_L6LoxtQVdPymYwwsUTAJGsi47Hn6APh8jB9juxBqaLguRf_ahZexssS7yVAxzjv9i5wd9aodsdCGzJ1Vtb_6miAEdkRrQgsOE9JSAthg




In [19]:
spotify = spotipy.Spotify(auth=token)
spotify.current_user_recently_played = types.MethodType(current_user_recently_played, spotify)



In [20]:
dave = spotify.current_user_recently_played(limit=50)
out_file = open("dave.json","w")
out_file.write(json.dumps(dave, sort_keys=True, indent=2))
out_file.close()

In [21]:
print(json.dumps(dave, sort_keys=True, indent=2))

{
  "cursors": {
    "after": "1578179067259",
    "before": "1577988320518"
  },
  "href": "https://api.spotify.com/v1/me/player/recently-played?limit=50",
  "items": [
    {
      "context": {
        "external_urls": {
          "spotify": "https://open.spotify.com/playlist/37i9dQZF1E39qQEuMzBnGd"
        },
        "href": "https://api.spotify.com/v1/playlists/37i9dQZF1E39qQEuMzBnGd",
        "type": "playlist",
        "uri": "spotify:playlist:37i9dQZF1E39qQEuMzBnGd"
      },
      "played_at": "2020-01-04T23:04:27.259Z",
      "track": {
        "album": {
          "album_type": "album",
          "artists": [
            {
              "external_urls": {
                "spotify": "https://open.spotify.com/artist/19zqV9DV3txjMUjHvltl2D"
              },
              "href": "https://api.spotify.com/v1/artists/19zqV9DV3txjMUjHvltl2D",
              "id": "19zqV9DV3txjMUjHvltl2D",
              "name": "Motion City Soundtrack",
              "type": "artist",
              "uri

In [61]:
Tracks = []

for i in range(50):
   Tracks.append(dave['items'][i]['track']['name'])

In [87]:
Artists = []

for i in range(50):
   Artists.append(dave['items'][i]['track']['artists'][0]['name'])

In [88]:
Albums = []


for i in range(50):
   Albums.append(dave['items'][i]['track']['album']['name'])

In [91]:
for (a, b, c) in itertools.izip_longest(Tracks, Artists, Album): 
    print (a, b, c) 

AttributeError: module 'itertools' has no attribute 'izip_longest'